- object detection: 
  A. classification
    - CNN 
  B. localization 
    - selective search:R-CNN, SPP-net, Fast-RCNN
      - 데이터 왜곡
      - 병목 현상
      - 이전에는 selective search, edge boxes 로 hand-engieered하고 병목현상으로 속도가 느림
      - resize,crop으로 데이터 왜곡
    - Faster: RPN 
     
### Faster-RCNN 
- 목표: 처음으로 RPN : **'물체'가 있을만한 '위치'region proposals를 학습**을 통해서 찾는 방법
- vectorization, GPU 사용해서 속도 향상
      - cls: object / not-object(background)
      - rgr: b-box (4 points)
  - convolution 특징: feature map에서  activation이 가장 큰 부분으로 물체 위치 '대략적으로 유추' 가능 
  - reduction: 각 비율,사이즈별로 9개의 anchor boxes(가장 많이 나올 물채 형태 크기) 값을 1개로 표현
-  IoU: positive sample: IoU >0.7, nagative sample: IoU < 0.3 
> 불균형 데이터 (imbalanced dataset) 카드사기비율이 전체 소유자에비해 적어서 학습이 잘 안됨

### RPN
    - multi-task loss: classification, regression 예측 :예.'물체'가 이 '사각형' 안에 있다.
      - convolution으로 학습통해서 featu분류객체, 위치를 동시에 반영가능 
    - linear regression : 정답과 실수간의 차이를 최소화
    - convolution 레이어로 들어오는 값이 feature잘 찾음, object 위치도 잘 찾음(공통적으로 사용해서 하나의 모델에 2개의 예측값)

- RPN 개선: 한번 더 학습 하지말고 위치 찾으면서 물체 맞추기 
- region proposal 여러개
> trends: 위치 찾으면서 어느 물체인지

# [AI 이노베이션 스퀘어] 12기 고급 시각반
- github: https://github.com/hysKim1/AI_Vision
    - [파이썬 공식문서]https://docs.python.org/ko/3/
    - [Numpy 공식문서]https://numpy.org/devdocs/
    - [ML]https://developers.google.com/machine-learning/crash-course/ml-intro?hl=ko
    - [DL]https://www.tensorflow.org/?hl=ko
      
---
* [Custom training](#Custom-training)
    * [GradientTape ](#GradientTape )
        * [context : with](#context- with)


        * [persistent](#persistent)
        * [watch](#watch)
    * [모델 학습](#모델-학습)
    * [학습 루프](#학습-루프)

context : with

---
## Custom training


1. tensorflow 사용해서 layer 커스터마이징
  - faster R-CNN
  - 객체지향 : 학습 가능한 레이어 만들기 위한 커스터마이징
    - __init__ 레이어 초기화 
    - build(input_shape): 가중치를 정의, 1번만 실행(self.built = True) 
    - call(x): 레이어의 논리의 핵심
        - build, call 호출
    - compute_output_shape(input_shape)
    - get_config


참고: https://keras.io/ko/layers/writing-your-own-keras-layers/

2. loss/objective func 커스터마이징 
- fit, train_on_batch, train 불가해서 만들어야함
  - 학습 : 실제,예측값 차이의 최소값을 갖도록 gradient descent 최적화 이론 구현 
    - optimizer :gradient descent 1차 미분하여 최소값 찾음
    - 상황마다 다른 gradient 방식 SGD,RMSProp, Adam
> - 책: Julia 실용 최적화 알고리즘(Algorithm for Optimization)

-  pytorch tutorial : Numpy vs pytorch/  tf (자동 미분,최적화,gpu 지원)

- Automatic differntitation in ML: Where we are and where we should be going
 by Google Brain
  - Numpy 비교해서 ML 프레임워크들은 일반성, 유연성을 포기
  - `tf.GradientTape` 자동미분
  - python 속도는 느리지만 간결
  - closure based
  - 미분기능 @tf.function

> - Pytorch Automatic differntitation in PyTorch(2017) by FB
   - **no tape**
   - operator overloading



### constant vs Variable
- 상수 미분 ->0 (기울기 0)

- python 상수의 개념 없음
- tensorflow : Variable, constant
  - immutable
 - 상수(변경 불가)

In [ ]:
import tensorflow as tf
a=tf.constant([1,2,3])

In [ ]:
a.assign([1,2,3]) 

AttributeError: ignored

In [ ]:
b=tf.Variable([4,5,6])

In [ ]:
b.assign([1,2,3])

<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

## tf.GradientTape
- 자동 미분: 후진 방식 자동 미분(reverse mode differentiation)을 사용해 테이프에 "기록된" 연산의 그래디언트를 계산
- context mngt 안에서 실행된 모든 연산 실행된 모든 미분한 결과값 저장(테이프(tape)에 기록)

- 참고 : https://www.tensorflow.org/guide/autodiff?hl=ko#%EA%B7%B8%EB%9E%98%EB%94%94%EC%96%B8%ED%8A%B8_%ED%85%8C%EC%9D%B4%ED%94%84

In [ ]:
x=tf.GradientTape
dir(x)  #__enter__, __exit__

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_pop_tape',
 '_push_tape',
 '_tf_api_names',
 '_tf_api_names_v1',
 'batch_jacobian',
 'gradient',
 'jacobian',
 'reset',
 'stop_recording',
 'watch',
 'watched_variables']

## context: with
- `__enter__`,`__exit__` 한 쌍으로 있어야 with **사용 가능**
  - `__enter__` 들어갈때 실행
  - `__exit__`  출력

In [ ]:
t=open('a.x','wb') 
dir(t) #__enter__, __exit__

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 '_dealloc_warn',
 '_finalizing',
 'close',
 'closed',
 'detach',
 'fileno',
 'flush',
 'isatty',
 'mode',
 'name',
 'raw',
 'read',
 'read1',
 'readable',
 'readinto',
 'readinto1',
 'readline',
 'readlines',
 'seek',
 'seekable',
 'tell',
 'truncate',
 'writable',
 'write',
 'writelines']

In [ ]:
with tf.GradientTape() as t:
''' 미분할 함수'''
  pass

In [ ]:
with tf.GradientTape() as t: #미분값 저장
  y=2*a  #a=tf.constant(1,2,3)

In [ ]:
with tf.GradientTape(persistent=True) as t: #미분값 저장
  y=2*b
  z=y**2

In [ ]:
dy_db=t.gradient(y,b)   #y를 b로 미분
dy_db     

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

In [ ]:
dy_db=t.gradient(z,b)   #z를 b로 미분
dy_db     

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 8., 16., 24.], dtype=float32)>

In [ ]:
dy_db=t.gradient(z,y)   #z를 y로 미분 : 식이 아닌 값으로 결과 나옴
dy_db   #상수로 계산함     

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 4.,  8., 12.], dtype=float32)>

- 식/변수가 아닌 값/상수로 결과가 나옴 

In [ ]:
y=2*b   #밖
with tf.GradientTape(persistent=True) as t: #미분값 저장
  z=y**2

In [ ]:
dy_db=t.gradient(z,b)  #z를 b로 미분
dy_db                  #Noen : with 내부에 계산해야함

1. tf 기본 dtype = 'float32'기반
2. `tf.Variable` 만 가능
3. with구문 내에서 1번만 실행 가능 (non-persistent)
  - 비슷한 예. iterator, generator 메모리 번지 초과하면 에러 
  
### gradient
- gradient 계산
- $\frac {dy}{dx}$: y함수를 x로 미분

constant 미분

In [ ]:
dy_da=t.gradient(y,a)  #한번만 실행

In [ ]:
dy_da  # Out 없음

In [ ]:
dy_da=t.gradient(y,a)  # non-persistent tapes

RuntimeError: ignored

Variable 미분

In [ ]:
with tf.GradientTape() as t: #미분값 저장
  y=2*b  #b=tf.Variable(1,2,3)

In [ ]:
dy_db=t.gradient(y,b)  #한번만 실행 non-persistent

In [ ]:
dy_db  # Out 없음

In [ ]:
dy_db=t.gradient(y,b) 

RuntimeError: ignored

In [ ]:
b=tf.Variable([1.,2,3]) #homogeneous -> type casting

In [ ]:
with tf.GradientTape() as t: #미분값 저장
  y=2*b 

In [ ]:
dy_db=t.gradient(y,b)  #한번만 실행 non-persistent
dy_db #결과값 나옴 

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

### watch 
`GradientTape.watch(constant)`
- 상수 사용 가능

In [ ]:
#1. 상수 가능 
a=tf.constant([1.0,2,3])

with tf.GradientTape() as t:
  t.watch(a)
  y=2*a

In [ ]:
dy_da=t.gradient(y,a)  
dy_da     #가능.

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

### persistent
`tf.GradientTape(persistent=True)`
- 동일한 연산에 대해  2번 이상 사용 가능(gradient() 메서드의 다중 호출을 허용)

In [ ]:
#2. 2번 이상 사용 가능  
with tf.GradientTape(persistent=True) as t: 
  t.watch(a)
  y=2*a

In [ ]:
dy_da=t.gradient(y,a)  
dy_da     #가능

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

In [ ]:
dy_da=t.gradient(y,a)  
dy_da     # 여러번 가능

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

- watched_accesible_ :trainable paramters 입력 들어오면 누적해서 추적 되는지 안되는 지체크.

### 제어
- 연산이 실행되는 순서대로 테이프에 기록되기 때문에 제어 가눙
- if while, for문 

In [ ]:
a=0
with tf.GradientTape(persistent=True) as t: 
  if a==0:
    
    y=2*b #밖에 두면 안됨
    z=y**2

In [ ]:
dy_db=t.gradient(z,b)  #z를 b로 미분
dy_db      #Noen 

TensorFlow에서 제공하는 기능
- Numpy 기반 사용하지 않음
- tensorflow + pandas
    - tensorflow 모델에 pandas 데이터 넣기 가능
- scikit-learn + keras 
    - GPU 사용 가능하도록

In [ ]:
import numpy as np
c=np.array([1.,2,3])

In [ ]:
with tf.GradientTape(persistent=True) as t: 
  y=2*c

In [ ]:
dy_dc=t.gradient(y,c)  #Numpy ndarray 불가
dy_dc     

AttributeError: ignored

## Custom training

[맞춤 학습](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough?hl=ko)
- 어떤 함수가 들어와도 미분 가능
- 처음부터 만들지 않고 다양한 실험 할 수 있도록 다양한 체크포인트를 둠.


### tf.data.Dataset 생성
- TensorFlow Dataset로  tensor 로 바꿔야함
    - tensorflow 에만 의존
    - 고급기능 이용 가능
>- Numpy기반을 바로 GPU호환 가능해지도록 시도

In [ ]:
import seaborn as sns
from sklearn import datasets
iris=sns.load_dataset('iris')#pandas

In [ ]:
tf.data.Dataset.from_generator      
tf.data.Dataset.from_tensor_slices #넘파이 포맷 받을 수 있음 
tf.data.Dataset.from_tensors

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.from_tensors>

In [ ]:
iris.iloc[:,:-1].values #넘파이

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
tf.data.Dataset.from_tensor_slices(iris.iloc[:,:-1].values) #텐서 (,4) input_shape

<TensorSliceDataset shapes: (4,), types: tf.float64>

In [ ]:
tf.data.Dataset.from_tensors(iris.iloc[:,:-1].values)      #텐서 (150,4)

<TensorDataset shapes: (150, 4), types: tf.float64>

In [ ]:
dir(iris) #__iter__ -> iterable -> iter()

['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__re

In [ ]:
iter(iris)

generator 만드는 방식 두가지 comprehension , yield

In [ ]:
tf.data.Dataset.from_generator( iter(iris), (tf.float32,tf.float32)) 

TypeError: ignored

In [ ]:
#generator 
def x():
  yield 1
  yield 2

In [ ]:
x() #callable

In [ ]:
def gen():
  for i in iris.iterrows(): 
    yield i 

In [ ]:
x= gen() #x callable 

In [ ]:
next(x) # 0th row

(0, sepal_length       5.1
 sepal_width        3.5
 petal_length       1.4
 petal_width        0.2
 species         setosa
 Name: 0, dtype: object)

In [8]:
tf.data.Dataset.from_generator( gen, (tf.float32,tf.float32))

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.float32, tf.float32)>

### 모델 학습
- fit, train_on_batch
    - callbacks 
    - multi-inputs, outputs 인 복잡한 모델에는  loss function 커스터마이징 하면 적용 불가한 경우가 많음.
    - eg. style transfer


- tf.GradientTape
    - 동적 사용 
    - tensorflow 에만 의존
    - 고급기능 이용

In [ ]:
# Sequential model
model=tf.keras.models.Sequential([
                              tf.keras.layers.Dense(10,activation='relu',
                                                    input_shape=(4,)),
                              tf.keras.layers.Dense(10,activation='relu'),
                              tf.keras.layers.Dense(3,activation='softmax')


]) 

In [ ]:
# 컴파일 학습 전략 - loss, optimizer, metrics 형태 고정
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

In [ ]:
model.fit(iris.iloc[:,:-1],iris.species,epochs=5)

Epoch 1/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1615
Epoch 2/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1476
Epoch 3/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1352
Epoch 4/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1259
Epoch 5/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1180


In [ ]:
#label encoding 
iris.species= iris.species.map({'setosa':0, 'versicolor':1,'virginica':2})

In [ ]:
model.fit(iris.iloc[:,:-1],iris.species,epochs=5)#pandas 연동 문제 없음

Epoch 1/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1109
Epoch 2/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1062
Epoch 3/5
5/5 [==============================] - 0s 2ms/step - loss: 1.1017
Epoch 4/5
5/5 [==============================] - 0s 2ms/step - loss: 1.0991
Epoch 5/5
5/5 [==============================] - 0s 2ms/step - loss: 1.0956


train_on_batch
- 한번 실행 시 singe batch size만큼 학습
- fit > train_on_batch
- for 돌리면 epochs 처럼 돌리기 가능
    - if 제어 가능
    -  값 변화 관찰 가능

In [ ]:
model.train_on_batch(iris.iloc[:,:-1],iris.species) 
#1 epoch 

1.0949578285217285

In [ ]:
for i in range(10): # 10번 학습
  print(model.train_on_batch(iris.iloc[:,:-1],iris.species))

1.0894548892974854
1.0890260934829712
1.0885772705078125
1.0881129503250122
1.0876123905181885
1.0870254039764404
1.0863898992538452
1.085587739944458
1.0845221281051636
1.0832358598709106


In [ ]:
dir(model) #weights, outputs...변화값 확인

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_los

In [ ]:
model.loss

'sparse_categorical_crossentropy'

In [ ]:
model.optimizer

In [ ]:
model.layers[0]

AttributeError: ignored

In [ ]:
model.fit(iris.iloc[:,:-1],iris.species,epochs=5,
          class_weight=) #balancing imbalanced dataset (class 비율 맞춰줌)

In [ ]:
a=tf.Variable([1.,2,3])

with tf.GradientTape() as t:
  y=a**2+ 2*a

In [ ]:
dy_da=t.gradient(y,a) #y를 a로 미분
dy_da

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([4., 6., 8.], dtype=float32)>

- optimizer: Gradient descent 방법
- loss: 학습 통해서 실제값,예측값 최소화 하는 weight(kernel, bias)구함

In [ ]:
optimizer=tf.keras.optimizers.Adam() #아담 방식으로 gradient descent

In [ ]:
optimizer.apply_gradients(zip(dy_da,a )) #인자 넣기  zip(grads, vars)쌍으로 묶음
#너무 단순해서 에러

TypeError: ignored

In [ ]:
#loss function: 들어온 데이터와 라벨 미분해서 weight(trainable, non-trainable) 찾기
loss= tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
loss()  #y_true, y_pred 차이 최소화

TypeError: ignored

In [ ]:
with tf.GradientTape() as t:     #실제값 예측값 최소화하도록 weights업데이트
  prediction=model(X_train)      #함수형 패러다임
  loss_=loss(y_train,prediction) # y - y pred 차이

In [ ]:
#예측 1.함수형 패러다임
model(iris.iloc[0,:-1].values[tf.newaxis]) # 예측 None ,4 

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.32492232, 0.3001846 , 0.37489307]], dtype=float32)>

In [ ]:
#예측 2.predict
model.predict(iris.iloc[0,:-1].values[tf.newaxis])

array([[0.32492232, 0.3001846 , 0.37489307]], dtype=float32)

In [ ]:
X_train =iris.iloc[:,:-1].values
y_train=iris.species.values

In [ ]:
model.weights

[<tf.Variable 'dense_6/kernel:0' shape=(4, 10) dtype=float32, numpy=
 array([[ 0.6379246 , -0.2855842 , -0.17817962, -0.24191003,  0.21669237,
         -0.15657482, -0.24792966, -0.50667083, -0.22154972,  0.5012336 ],
        [-0.17175655, -0.00971794, -0.5302    ,  0.55513346, -0.09684049,
          0.24519277,  0.52446806, -0.05733848, -0.30281532,  0.37406987],
        [-0.4123497 ,  0.14434832, -0.61100405,  0.28042528, -0.61468005,
          0.06093141,  0.2083592 , -0.37035027, -0.29403198, -0.02872974],
        [-0.42922887, -0.04198253, -0.31133798,  0.25882214,  0.14789581,
          0.10624892, -0.3545277 ,  0.3655163 , -0.2881619 , -0.11225794]],
       dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(10,) dtype=float32, numpy=
 array([ 0.05036322,  0.        ,  0.        ,  0.0519118 , -0.0389467 ,
        -0.02074672, -0.04862819,  0.        ,  0.        , -0.04847603],
       dtype=float32)>,
 <tf.Variable 'dense_7/kernel:0' shape=(10, 10) dtype=float32, numpy=
 arr

In [ ]:
model.trainable_variables

[<tf.Variable 'dense_6/kernel:0' shape=(4, 10) dtype=float32, numpy=
 array([[ 0.6379246 , -0.2855842 , -0.17817962, -0.24191003,  0.21669237,
         -0.15657482, -0.24792966, -0.50667083, -0.22154972,  0.5012336 ],
        [-0.17175655, -0.00971794, -0.5302    ,  0.55513346, -0.09684049,
          0.24519277,  0.52446806, -0.05733848, -0.30281532,  0.37406987],
        [-0.4123497 ,  0.14434832, -0.61100405,  0.28042528, -0.61468005,
          0.06093141,  0.2083592 , -0.37035027, -0.29403198, -0.02872974],
        [-0.42922887, -0.04198253, -0.31133798,  0.25882214,  0.14789581,
          0.10624892, -0.3545277 ,  0.3655163 , -0.2881619 , -0.11225794]],
       dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(10,) dtype=float32, numpy=
 array([ 0.05036322,  0.        ,  0.        ,  0.0519118 , -0.0389467 ,
        -0.02074672, -0.04862819,  0.        ,  0.        , -0.04847603],
       dtype=float32)>,
 <tf.Variable 'dense_7/kernel:0' shape=(10, 10) dtype=float32, numpy=
 arr

In [ ]:
model.trainable_weights

[<tf.Variable 'dense_6/kernel:0' shape=(4, 10) dtype=float32, numpy=
 array([[ 0.6379246 , -0.2855842 , -0.17817962, -0.24191003,  0.21669237,
         -0.15657482, -0.24792966, -0.50667083, -0.22154972,  0.5012336 ],
        [-0.17175655, -0.00971794, -0.5302    ,  0.55513346, -0.09684049,
          0.24519277,  0.52446806, -0.05733848, -0.30281532,  0.37406987],
        [-0.4123497 ,  0.14434832, -0.61100405,  0.28042528, -0.61468005,
          0.06093141,  0.2083592 , -0.37035027, -0.29403198, -0.02872974],
        [-0.42922887, -0.04198253, -0.31133798,  0.25882214,  0.14789581,
          0.10624892, -0.3545277 ,  0.3655163 , -0.2881619 , -0.11225794]],
       dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(10,) dtype=float32, numpy=
 array([ 0.05036322,  0.        ,  0.        ,  0.0519118 , -0.0389467 ,
        -0.02074672, -0.04862819,  0.        ,  0.        , -0.04847603],
       dtype=float32)>,
 <tf.Variable 'dense_7/kernel:0' shape=(10, 10) dtype=float32, numpy=
 arr

- 관례상 학습에서는 trainable_variables 변수 사용

In [ ]:
model.trainable_variables is model.trainable_weights #다른 객체로 만들어짐 

False

In [ ]:
model.trainable_variables == model.trainable_weights #동일한 값 

True

In [ ]:
grad=t.gradient(loss_, model.trainable_variables)

In [ ]:
optimizer.apply_gradients(zip(grad, model.trainable_variables)) #너무 작아서임 ?

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

- 함수형 패러다임 : calll 객체 인스턴스 () 
- compile, fit 수동으로 함수로 만듦
    - 똑같은 값으로 gradient 안함
    - train할때마다 loss,정확도 값 계산

In [ ]:
#accuracy 변하는 정도 
train_loss= tf.keras.metrics.Mean()
train_acc=tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
def train(X_train,y_train):
        #함수형 패러다임

    with tf.GradientTape() as t: 
        prediction=model(X_train)
        loss_=loss(y_train,prediction) 
        
    grad=t.gradient(loss_, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))
    
    train_loss(loss_)
    train_acc(y_train, prediction)

In [ ]:
@tf.function   #decorator: 넘파이를 텐서에 쓰는 함수로 변경해서 효율적으로 사용할 수 있도록 함 

def train(X_train,y_train):
  with tf.GradientTape() as t:  #실제값 예측값 최소화하도록 weights업데이트
    prediction=model(X_train)
    loss_=loss(y_train,prediction) # y - y pred 차이
  grad=t.gradient(loss_, model.trainable_variables)
  optimizer.apply_gradients(zip(grad, model.trainable_variables))
  #  optimizer.apply_gradients([grad, model.trainable_variables])
  train_loss(loss_)
  train_acc(y_train, prediction)

### 학습 루프
- epochs= for 

In [ ]:
for i in range(10):  #10 epochs
  train(X_train,y_train)
  print(i+1, train_acc.result()) # 학습 결과 출력

1 tf.Tensor(0.3979487, shape=(), dtype=float32)
2 tf.Tensor(0.39698112, shape=(), dtype=float32)
3 tf.Tensor(0.39604938, shape=(), dtype=float32)
4 tf.Tensor(0.39515153, shape=(), dtype=float32)
5 tf.Tensor(0.3942857, shape=(), dtype=float32)
6 tf.Tensor(0.3934503, shape=(), dtype=float32)
7 tf.Tensor(0.3926437, shape=(), dtype=float32)
8 tf.Tensor(0.39186442, shape=(), dtype=float32)
9 tf.Tensor(0.3911111, shape=(), dtype=float32)
10 tf.Tensor(0.39038253, shape=(), dtype=float32)


In [ ]:
train_acc.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.41053763>

In [ ]:
# trainable_weights 
@tf.function   #decorator: 넘파이를 텐서로 변경해서 효율적으로 사용할수있도록 함 
  #함수형 패러다임
def train(X_train,y_train):
  with tf.GradientTape() as t:  #실제값 예측값 최소화하도록 weights업데이트
    prediction=model(X_train)
    loss_=loss(y_train,prediction) # y - y pred 차이
 #모델 최적화
  grad=t.gradient(loss_, model.trainable_weights)
  optimizer.apply_gradients(zip(grad, model.trainable_weights))
    
  train_loss(loss_)

  train_acc(y_train, prediction)

In [ ]:
for i in range(10):
  train(X_train,y_train)
  print(i+1, train_acc.result()) 

1 tf.Tensor(0.3896774, shape=(), dtype=float32)
2 tf.Tensor(0.3888889, shape=(), dtype=float32)
3 tf.Tensor(0.388125, shape=(), dtype=float32)
4 tf.Tensor(0.38738462, shape=(), dtype=float32)
5 tf.Tensor(0.38666666, shape=(), dtype=float32)
6 tf.Tensor(0.38597015, shape=(), dtype=float32)
7 tf.Tensor(0.3852941, shape=(), dtype=float32)
8 tf.Tensor(0.38454106, shape=(), dtype=float32)
9 tf.Tensor(0.38380954, shape=(), dtype=float32)
10 tf.Tensor(0.3830986, shape=(), dtype=float32)


- 원하는 결과에 따라서 다양한 그래프 만듦